In [1]:
# Import libraries
import os
import re
import random
import pickle
import subprocess
import numpy as np
import pandas as pd
import datetime as dt

from tqdm import tqdm
from datetime import datetime
from collections import Counter

# 1. Setup concept extractors

Some options were [MetaMap](https://metamap.nlm.nih.gov/) and [spaCy](https://spacy.io/). 

[MetaMap](https://metamap.nlm.nih.gov/) is specific to recognizing UMLS concepts. There is a [Python wrapper](https://github.com/AnthonyMRios/pymetamap), but known to be slow and bad.

[spaCy](https://spacy.io/) is a popular NLP Python package with an extensive library for named entity recognition. It has a wide variety of [extensions](https://spacy.io/universe) and models to choose from. We're going with the following.

* [scispaCy](https://spacy.io/universe/project/scispacy) contains spaCy models for processing biomedical, scientific or clinical text. It seems easy to use and has a wide variety of concepts it can recognize, including UMLS, RxNorm, etc.

* [negspaCy](https://spacy.io/universe/project/negspacy) identifies negations using some extension of regEx. Probably useful for things like, "this pt is diabetic" v. "this pt is not diabetic." [todo: negation identification of medspacy might be better, https://github.com/medspacy/medspacy]

* [Med7](https://github.com/kormilitzin/med7) is a model trained for recognizing entities in prescription text, e.g. identifies drug name, dosage, duration, etc., which could be useful stuff to check for conflicts. 

We're going with spaCy for this.. and coming up with a coherent way to integrate entities picked up by these three extensions/models.

## i) Installations

In [2]:
import sys; sys.executable

'/opt/conda/envs/opennotes/bin/python'

In [3]:
import spacy
import scispacy

from pprint import pprint
from collections import OrderedDict

from spacy import displacy
# from scispacy.abbreviation import AbbreviationDetector # UMLS already contains abbrev. detect
from scispacy.umls_linking import UmlsEntityLinker

# should be 2.3.5 and >=0.3.0
spacy.__version__, scispacy.__version__

('2.3.5', '0.3.0')

## ii) Setting up the model

The model is used to form word/sentence embeddings for the NER task. Thus, it's important to choose model that has been tuned for our specific use case (e.g. clinical text, prescription information) so the embeddings are useful for naming the entity.

[Note to self:] one potential idea to look into if we have time remaining, something about using custom model for spacy pipeline (could we do smth with the romanov models since they've been trained specifically for conflict detection?) -- https://spacy.io/usage/v3

### a) scispaCy

For scispaCy, we set up one of their models that has been trained on biomedical data. Other models can be found [here](https://allenai.github.io/scispacy/). 

We load two models since we will be linking different entity linkers (knowledge bases that link text to named entites) later.

In [4]:
## uncomment to install model if not already installed
# !/opt/conda/envs/opennotes/bin/python -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.5/en_core_sci_sm-0.2.5.tar.gz

In [5]:
# for umls (general biomedical concepts)
umls_nlp   = spacy.load("en_core_sci_sm")

# for rxnorm (prescriptions)
rxnorm_nlp = spacy.load("en_core_sci_sm")

### b) Med7

For Med7, we set up their model that has been trained specifically for NER of medication-related concepts: dosage, drug names, duration, form, frequency, route of administration, and strength. The model is trained on MIMIC-III, so it should work well for us.

In [6]:
# # installs Med7 model
# !pip install https://www.dropbox.com/s/xbgsy6tyctvrqz3/en_core_med7_lg.tar.gz?dl=1

In [7]:
med7_nlp = spacy.load("en_core_med7_lg")

## iii) Adding an entity linker

The EntityLinker is a spaCy component that links to a knowledge base. The linker compares words with the concepts in the specified knowledge base (e.g. scispaCy's UMLS does some form of character overlap-based nearest neighbor search, has option to resolve abbreviations first).

[Note: Entities generally get resolved to a list of different entities. This [blog post](http://sujitpal.blogspot.com/2020/08/disambiguating-scispacy-umls-entities.html) describes one potential way to disambiguate this by figuring out "most likely" set of entities. Gonna start off with just resolving to the 1st entity tho... hopefully that's sufficient.]

### a) scispaCy

#### UMLS Linker

UMLS linker maps entities to the UMLS concept. Main parts we'll be interested in are: semantic type and concept (mainly the common name, maybe the CUI might become important later).

* _Semantic type_ is the broader category that the entity falls under, e.g. disease, pharmacologic substance, etc. See [this](https://metamap.nlm.nih.gov/Docs/SemanticTypes_2018AB.txt) for a full list.

* _Concepts_ refer to the more fundamental entity itself, e.g. pneumothorax, ventillator, etc. Many concepts can fall under a semantic type.

More info on `UmlsEntityLinker` ([source code](https://github.com/allenai/scispacy/blob/4ade4ec897fa48c2ecf3187caa08a949920d126d/scispacy/linking.py#L9))

See source code for `.jsonl` file with the knowledge base.

In [8]:
from scispacy.umls_linking import UmlsEntityLinker

# abbreviation_pipe = AbbreviationDetector(nlp) # automatically included with UMLS linker
# nlp.add_pipe(abbreviation_pipe)
umls_linker = UmlsEntityLinker(k=10,                          # number of nearest neighbors to look up from
                               threshold=0.7,                 # confidence threshold to be added as candidate
                               max_entities_per_mention=1,    # number of entities returned per concept (todo: tune)
                               filter_for_definitions=False,  # no definition is OK
                               resolve_abbreviations=True)    # resolve abbreviations before linking
umls_nlp.add_pipe(umls_linker)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


#### RxNorm Linker

RxNorm linker maps entities to RxNorm, an ontology for clinical drug names. It contains about 100k concepts for normalized names for clinical drugs. It is comprised of several other drug vocabularies commonly used in pharmacy management and drug interaction, including First Databank, Micromedex, and the Gold Standard Drug Database.

More info on `RxNorm` ([NIH page](https://www.nlm.nih.gov/research/umls/rxnorm/index.html), [source code](https://github.com/allenai/scispacy/blob/2290a80cfe0948e48d8ecfbd60064019d57a6874/scispacy/linking_utils.py#L120))

See source code for `.jsonl` file with the knowledge base.

In [9]:
from scispacy.linking import EntityLinker

# rxnorm_linker = EntityLinker(resolve_abbreviations=True, name="rxnorm")
rxnorm_linker = EntityLinker(k=10,                          # number of nearest neighbors to look up from
                             threshold=0.7,                 # confidence threshold to be added as candidate
                             max_entities_per_mention=1,    # number of entities returned per concept (todo: tune)
                             filter_for_definitions=False,  # no definition is OK
                             resolve_abbreviations=True,    # resolve abbreviations before linking
                             name="rxnorm")                 # RxNorm ontology

rxnorm_nlp.add_pipe(rxnorm_linker)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


### b) Med7 

No need for entity linker

### c) Negspacy [TODO]

# 2. Setup data structures

## Categorizing type of conflict

The first larger task is to categorize by the type of conflict to check for since our method will likely be different (at least for the rule based). We wrote up a short list [here](https://docs.google.com/document/d/1fEBk0JHeyQWshYWW5w_VTkaYyRfm9MBxJ9DAGoVa8Yw/edit?usp=sharing). 

To do this, we're using the semantic type that is identified by the UMLS linker. Here's a table of the semantic types we're filtering for, and which conflict they'll be used for.

Here's a [full list](https://metamap.nlm.nih.gov/Docs/SemanticTypes_2018AB.txt) of semantic types. You can look up definitions of semantic types [here](http://linkedlifedata.com/resource/umls-semnetwork/T033).

| Conflict | Semantic Type |
| --- | ----------- |
| Diagnoses-related errors | Disease or Syndrome (T047), Diagnostic Procedure(T060) |
| Inaccurate description of medical history (symptoms) | Sign or Symptom (T184) |
| Inaccurate description of medical history (operations) | Therapeutic or Preventive Procedure (T061) |
| Inaccurate description of medical history (other) | [all of the above and below] |
| Medication or allergies | Clinical Drug (T200), Pharmacologic Substance (T121) |
| Test procedures or results | Laboratory Procedure (T059), Laboratory or Test Result (T034) | 


For clarity, the concepts we'll keep from the UMLS linker are anything falling into these semantic types (which we will then categorize by type of conflict using the table above):

* T047 - Disease or Syndrome
* T121 - Pharmacologic Substance
* T023 - Body Part, Organ, or Organ Component
* T061 - Therapeutic or Preventive Procedure 
* T060 - Diagnostic Procedure
* T059 - Laboratory Procedure
* T034 - Laboratory or Test Result 
* T184 - Sign or Symptom 
* T200 - Clinical Drug

We'll store this info into a dictionary now.

<!-- Some useful def's 
Finding - 
That which is discovered by direct observation or measurement of an organism attribute or condition, including the clinical history of the patient. The history of the presence of a disease is a 'Finding' and is distinguished from the disease itself.  -->

In [10]:
SEMANTIC_TYPES = ['T047', 'T121', 'T023', 'T061', 'T060', 'T059', 'T034', 'T184', 'T200']
SEMANTIC_NAMES = ['Disease or Syndrome', 'Pharmacologic Substance', 'Body Part, Organ, or Organ Component', \
                  'Therapeutic or Preventive Procedure', 'Diagnostic Procedure', 'Laboratory Procedure', \
                  'Laboratory or Test Result', 'Sign or Symptom', 'Clinical Drug']
SEMANTIC_TYPE_TO_NAME = dict(zip(SEMANTIC_TYPES, SEMANTIC_NAMES))

SEMANTIC_TYPE_TO_NAME

{'T047': 'Disease or Syndrome',
 'T121': 'Pharmacologic Substance',
 'T023': 'Body Part, Organ, or Organ Component',
 'T061': 'Therapeutic or Preventive Procedure',
 'T060': 'Diagnostic Procedure',
 'T059': 'Laboratory Procedure',
 'T034': 'Laboratory or Test Result',
 'T184': 'Sign or Symptom',
 'T200': 'Clinical Drug'}

In [11]:
CONFLICT_TO_SEMANTIC_TYPE = {
    "diagnosis": {'T047', 'T060'},
    "med_history_symptom": {'T184'},
    "med_history_operation": {'T061'},
    "med_history_other": set(SEMANTIC_TYPES),
    "med_allergy": {'T200', 'T121'},
    "test_results": {'T059', 'T034'}
}

CONFLICT_TO_SEMANTIC_TYPE

{'diagnosis': {'T047', 'T060'},
 'med_history_symptom': {'T184'},
 'med_history_operation': {'T061'},
 'med_history_other': {'T023',
  'T034',
  'T047',
  'T059',
  'T060',
  'T061',
  'T121',
  'T184',
  'T200'},
 'med_allergy': {'T121', 'T200'},
 'test_results': {'T034', 'T059'}}

In [12]:
from data_structures import Patient,\
                            Note, PrescriptionOrders, LabResults,\
                            Sentence, Prescription, Lab

In [13]:
# from importlib import reload # python 2.7 does not require this
# import data_structures
# reload(data_structures)
# from data_structures import Patient,\
#                             Note, PrescriptionOrders, LabResults,\
#                             Sentence, Prescription, Lab

# 3. Load and process data

In [14]:
# Load MIMIC tables
notes_df  = pd.read_csv('NOTEEVENTS.csv.gz',    compression='gzip', error_bad_lines=False)
drug_df   = pd.read_csv('PRESCRIPTIONS.csv.gz', compression='gzip', error_bad_lines=False)
lab_df    = pd.read_csv('LABEVENTS.csv.gz',     compression='gzip', error_bad_lines=False)
d_lab_df  = pd.read_csv('D_LABITEMS.csv.gz',    compression='gzip', error_bad_lines=False)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Updated script for processing HADM ID's with consecutive physician notes (does not count the autosaves)

In [15]:
# Load HADM ID's with consecutive physician notes
if os.path.exists("hadm_ids.pkl"):
    with open("hadm_ids.pkl", "rb") as f:
        hadm_ids = pickle.load(f)
else:
    hadm_ids = []
    for hadm_id in tqdm(notes_df.HADM_ID.unique()):
        hadm_data = notes_df.loc[notes_df.HADM_ID == hadm_id]
        hadm_phys_notes = hadm_data.loc[hadm_data.CATEGORY == "Physician "]

        if len(hadm_phys_notes.CHARTTIME.unique()) > 1: # ensure > 1 unique notes (not counting autosave)
            hadm_ids.append(hadm_id)

    with open("hadm_ids.pkl", "wb") as f:
        pickle.dump(hadm_ids, f)
        
print(f"There are {len(hadm_ids)} patients with consecutive physician notes.")

There are 8158 patients with consecutive physician notes.


# 4. Defining Pair Generation Code

In [16]:
def is_comparable_type(data_i, data_j):
    """ We only want to compare note-to-note OR note-to-structured data. 
    
    Comparable types:
    - sentence v. sentence
    - sentence v. prescription
    - sentence v. lab
    
    Uncomparable types:
    - lab v. lab 
    - lab v. prescription
    - prescription v. prescription
    """
    return (data_i.type == "sentence"     and data_j.type == "sentence") or \
           (data_i.type == "sentence"     and data_j.type == "prescription") or \
           (data_i.type == "prescription" and data_j.type == "sentence") or \
           (data_i.type == "sentence"     and data_j.type == "lab") or \
           (data_i.type == "lab"          and data_j.type == "sentence")

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_data_pairs(pat):
    processed_pairs = []  # for dataframe + csv
    data_inst_pairs = []  # for pipeline, list of tuples: ((Data 1, Data 2), label)
    pair_idx = 0

    # Iterate over all of the patient's DailyData instances (e.g. note, prescription order, lab results for same day)
    ## pat.dailydata = {[date]: [DailyData instance from that date], ...}
    for day, pat_dailydatas in pat.dailydata.items(): # pat_dailydatas is list of all DailyData instances for `day`
        print(f"********** Processing data for {day} **********")
        # Collect all the daily datas (note, prescription orders, lab results) for current day
        current_dds = []
        current_dds_features = []
        current_dds_txts = []
        current_dds_sem_types = []
        current_dds_sem_names = []
        for dd in pat_dailydatas: # iterating over DailyData instances, e.g. dd=physician note taken on `day`
            current_dds.extend(dd.datas)
            current_dds_features.extend(dd.datas_features)
            current_dds_txts.extend(dd.datas_txts)
            current_dds_sem_types.extend(dd.datas_semantic_types)
            current_dds_sem_names.extend(dd.datas_semantic_names)

        current_dds           = np.array(current_dds)
        current_dds_features  = np.array(current_dds_features)
        current_dds_txts      = np.array(current_dds_txts)
        current_dds_sem_types = np.array(current_dds_sem_types)
        current_dds_sem_names = np.array(current_dds_sem_names)

        # extract similar sentences for each semantic type
        for sem_type in SEMANTIC_TYPES:
            # data for this semantic type
            sem_type_bools   = [sem_type in x for x in current_dds_sem_types]
            sem_type_indices = np.where(sem_type_bools)[0]
            indices_map = dict(
                            zip(range(len(sem_type_indices)), 
                                sem_type_indices)
                          )  # maps regular indices in sem_type_current_dds_* lists to indices in current_dds_* lists

            sem_type_current_dds           = current_dds[sem_type_indices]
            sem_type_current_dds_features  = current_dds_features[sem_type_indices]
            sem_type_current_dds_txts      = current_dds_txts[sem_type_indices]
            sem_type_current_dds_sem_types = current_dds_sem_types[sem_type_indices]
            sem_type_current_dds_sem_names = current_dds_sem_names[sem_type_indices]

            # current_dds_featuresfor features (umls + rxnorm concepts)
            vectorizer = CountVectorizer()
            corpus = list(map(lambda x: ' '.join(x), sem_type_current_dds_features))
            if len(corpus) == 0: # skip rest if no candidate sentences exist
                continue
            X = vectorizer.fit_transform(corpus)
            X = X.toarray()

            # get cosine similarity using umls + rxnorm concepts
            similarity = cosine_similarity(X)     # larger=more similar
            sim_is, sim_js = np.where(similarity>0.5) # all pairs with at least 0.5 similarity

            for i, j in zip(sim_is, sim_js):
                data_i = sem_type_current_dds[i]
                data_j = sem_type_current_dds[j]
                # removing same sentence pairs, checking dates
                if i>j and is_comparable_type(data_i, data_j):
                    print(f"***** PAIR INDEX {pair_idx} *****")
                    print(f"Cosine similarity: {similarity[i, j]}")
                    print(f"----- Data i -----")
                    print(f">> Time: {data_i.time}\n" +\
                          f">> Type: {data_i.type}\n" +\
                          f">> Concepts: {data_i.features}\n" +\
                          f">> {data_i.txt}")
                    print(f"----- Data j -----")
                    print(f">> Time: {data_j.time}\n" +\
                          f">> Type: {data_j.type}\n" +\
                          f">> Concepts: {data_j.features}\n" +\
                          f">> {data_j.txt}")
                    print("**********************************")

                    # save
                    processed_pairs.append([data_i.txt,      data_j.txt, \
                                            data_i.time,     data_j.time, \
                                            data_i.type,     data_j.type, \
                                            data_i.features, data_j.features, \
                                            similarity[i, j], SEMANTIC_TYPE_TO_NAME[sem_type]])
            #                                 SEMANTIC_TYPE_TO_NAME[semantic_type]])

                    data_inst_pairs.append(((data_i, data_j), None))
                    pair_idx += 1

    ###############
    #### Final ####
    ###############        
    df = \
    pd.DataFrame(np.array(processed_pairs), \
                 columns=["sentence 1", "sentence 2", \
                          "time 1", "time 2", \
                          "type 1", "type 2", \
                          "concepts 1", "concepts 2", \
                          "cosine similarity", "semantic type"])
    
    return df, data_inst_pairs

# 5. Defining Feature Generation Code

In [18]:
is_diagnosis = lambda stype: stype in CONFLICT_TO_SEMANTIC_TYPE['diagnosis']
is_med       = lambda stype: stype in CONFLICT_TO_SEMANTIC_TYPE['med_allergy']
is_test      = lambda stype: stype in CONFLICT_TO_SEMANTIC_TYPE['test_results']

def get_conflict_type(s1, s2):
    conflict_types = []
    
    is_diag_s1 = any([is_diagnosis(stype) for stype in s1.semantic_types])
    is_diag_s2 = any([is_diagnosis(stype) for stype in s2.semantic_types])
    if is_diag_s1 and is_diag_s2: conflict_types.append("diagnosis")

    is_med_s1 = any([is_med(stype) for stype in s1.semantic_types])
    is_med_s2 = any([is_med(stype) for stype in s2.semantic_types])
    if is_med_s1 and is_med_s2: conflict_types.append("med")

    is_test_s1 = any([is_test(stype) for stype in s1.semantic_types])
    is_test_s2 = any([is_test(stype) for stype in s2.semantic_types])
    if is_test_s1 and is_test_s2: conflict_types.append("test")
        
    return conflict_types

In [19]:
# if number of neg tokens is equal, return 0; otherwise, return 1
def check_if_number_neg_equal_umls(s1, s2):
    sent_doc_1_umls = s1.umls_doc    # "Doc" output for UMLS 
    sent_doc_2_umls = s2.umls_doc    # "Doc" output for UMLS
    
    negation_tokens_1 = [tok for tok in sent_doc_1_umls if tok.dep_ == 'neg']
    negation_tokens_2 = [tok for tok in sent_doc_2_umls if tok.dep_ == 'neg']

    if len(negation_tokens_1) != len(negation_tokens_2):
        return 1
    else:
        return 0

In [20]:
"""
Get dependency tokens based on UMLS ontology
"""
def create_dep_encoding(s1, s2):

    sent_doc_1 = s1.umls_doc
    sent_doc_2 = s2.umls_doc

    dep_child_1 = []
    dep_child_2 = []

    for token in sent_doc_1:
        if token.dep_ == "ROOT": 
            index_to_check = token.i
            dep_list_1 = [token.text for token in sent_doc_1[index_to_check].children if token.dep_ != "punct"]
            dep_child_1.append(sent_doc_1[index_to_check].text)
            dep_child_1.extend(dep_list_1)

    for token in sent_doc_2:
        if token.dep_ == "ROOT":
            index_to_check = token.i
            dep_list_2 = [token.text for token in sent_doc_2[index_to_check].children if token.dep_ != "punct"]
            dep_child_2.append(sent_doc_2[index_to_check].text)
            dep_child_2.extend(dep_list_2)

    dep_sent_1 = list_to_string(dep_child_1)
    dep_sent_2 = list_to_string(dep_child_2)

    dep_doc_1 = umls_nlp(dep_sent_1)
    dep_doc_2 = umls_nlp(dep_sent_2)
    similarity = dep_doc_1.similarity(dep_doc_2)
    return similarity

def list_to_string(list1):
    str1 = ""
    for element in list1:
        str1 += " " + element

    return str1

In [21]:
"""
Given Sentence instances s1, s2
check if they share a shared concept feature (UMLS and RxNorm concepts) 
if they do not share a concept, return 1; otherwise, return 0
"""
def check_shared_feature_umls(s1,s2):
    #{'Scanning'}
    s1_features = s1.features
    s2_features = s2.features
    
    # check if share feature in common
    shared_feature = list(set(s1_features) & set(s2_features))
    
    # do not share concept
    if shared_feature != []:
        return 1
    else: 
        return 0

In [22]:
"""
Med7 (prescription) entities
# if not talking about same DRUG -> return 0
# if same DRUG but other info different -> return 1
"""
def check_shared_feature_med7(s1,s2):
    
    # [('2U', 'DOSAGE'), ('PRBC', 'DRUG')]
    s1_features = s1.med7_entities
    s2_features = s2.med7_entities
    
    # get drug names for each sentence
    s1_names = [name for (name, word_type) in s1_features if word_type == "DRUG"]
    s2_names = [name for (name, word_type) in s2_features if word_type == "DRUG"]
    
    # check if drug name is in common
    shared_drug = list(set(s1_names) & set(s2_names))
    
    # share drug name, but linkers are different
    if shared_drug != [] and s1_features != s2_features:
        return 1
    else: 
        return 0

In [23]:
"""
Use above methods to get features for data
s1, s2 are Sentence instances
"""
def get_feature_df_mimic(s1, s2, hadm_id, pair_id):
    sentence_1 = s1.txt
    sentence_2 = s2.txt
    
    # check negation in docs
    neg_check_umls = check_if_number_neg_equal_umls(s1, s2)

    # check shared features in UMLS and Med7
    check_umls = check_shared_feature_umls(s1,s2)
    check_med7 = check_shared_feature_med7(s1,s2)
    
    # find dependent children similarity
    dep_similarity = create_dep_encoding(s1, s2)
    
    # get conflict type -- returns list 
    conflict_type = get_conflict_type(s1, s2)
    
    sentence_info = [neg_check_umls, check_umls, check_med7, \
                     sentence_1, sentence_2, dep_similarity, \
                     hadm_id, conflict_type, pair_id]
    
    return sentence_info

# 6. Predicting Contradiction for Anvil

## Load Hybrid Model

In [24]:
from typing import Dict, Tuple
from sklearn.base import BaseEstimator

class RuleAugmentedEstimator(BaseEstimator):
  """
  Augments sklearn estimators with deterministic rule-based logic.
  """

  def __init__(self, base_model: BaseEstimator, rules: Dict, **base_params):
      """
      Initializes the rule-augmented estimator by supplying underlying sklearn estimator
      and hard-coded rules.

      Args:
        base_model: underlying sklearn estimator.
          Must implement fit and predict method.
        rules: hard coded rules in format of dictionary,
          with keys being the pandas dataframe column name, 
          and values being a tuple in the following form: 
          (comparison operator, value, return value)

          Acceptable comparison operators are: 
          "=", "<", ">", "<=", ">="

          Example:
                
                {"House Type": [
                    ("=", "Penthouse", 1.0),
                    ("=", "Shack", 0.0)
                  ],
                  "House Price": [
                      ("<", 1000.0, 0.0),
                      (">=", 500000.0, 1.0)
                ]}
        **base_params: Optional keyword arguments which will be passed on
            to the base_model.

      """
      self.rules = rules
      self.base_model = base_model
      self.base_model.set_params(**base_params)
      self.outcome_range = [0,1]

  def __repr__(self):
      return "Rule Augmented Estimator:\n\n\t Base Model: {}\n\t Rules: {}".format(self.base_model, self.rules)

  def __str__(self):
      return self.__str__

  def _get_base_model_data(self, X: pd.DataFrame, y: pd.Series) -> Tuple[pd.DataFrame, pd.Series]:
      """
      Filters the training data for data points not affected by the rules.
      """
      train_x = X

      for category, rules in self.rules.items():

          if category not in train_x.columns.values: continue

          for rule in rules:

              if rule[0] == "=":
                  train_x = train_x.loc[train_x[category] != rule[1]]

              elif rule[0] == "<":
                  train_x = train_x.loc[train_x[category] >= rule[1]]

              elif rule[0] == ">":
                  train_x = train_x.loc[train_x[category] <= rule[1]]

              elif rule[0] == "<=":
                  train_x = train_x.loc[train_x[category] > rule[1]]

              elif rule[0] == ">=":
                  train_x = train_x.loc[train_x[category] < rule[1]]

              else:
                  print("Invalid rule detected: {}".format(rule))
              
      indices = train_x.index.values
      train_y = y.iloc[indices]
      
      train_x = train_x.reset_index(drop=True)
      train_y = train_y.reset_index(drop=True)
      
      return train_x, train_y   

  def fit(self, X: pd.DataFrame, y: pd.Series, **kwargs):
      """Fits the estimator to the data.
      
      Fits the estimator to the data, only training the underlying estimator
      on data which isn't affected by the hard-coded rules.
      
      Args:
          X: The training feature data.
          y: The training label data.
          **kwargs: Optional keyword arguments passed to the underlying
          estimator's fit function.
              
      """
      train_x, train_y = self._get_base_model_data(X, y)
      self.base_model.fit(train_x, train_y, **kwargs)

  def predict(self, X: pd.DataFrame) -> np.array:
      """Gets predictions for the provided feature data.
      
      The predicitons are evaluated using the provided rules wherever possible
      otherwise the underlying estimator is used.
      
      Args:
          X: The feature data to evaluate predictions for.
      
      Returns:
          np.array: Evaluated predictions.
      """
      
      p_X = X.copy()
      p_X['prediction'] = np.nan

      for category, rules in self.rules.items():

          if category not in p_X.columns.values: continue

          for rule in rules:

              if rule[0] == "=":
                  p_X.loc[p_X[category] == rule[1], 'prediction'] = rule[2]

              elif rule[0] == "<":
                  p_X.loc[p_X[category] < rule[1], 'prediction'] = rule[2]

              elif rule[0] == ">":
                  p_X.loc[p_X[category] > rule[1], 'prediction'] = rule[2]

              elif rule[0] == "<=":
                  p_X.loc[p_X[category] <= rule[1], 'prediction'] = rule[2]

              elif rule[0] == ">=":
                  p_X.loc[p_X[category] >= rule[1], 'prediction'] = rule[2]

              else:
                  print("Invalid rule detected: {}".format(rule))

      if len(p_X.loc[p_X['prediction'].isna()].index != 0):

          base_X = p_X.loc[p_X['prediction'].isna()].copy()
          base_X.drop('prediction', axis=1, inplace=True)
          p_X.loc[p_X['prediction'].isna(), 'prediction'] = self.base_model.predict(base_X)

      return p_X['prediction'].values
    
  def get_params(self, deep: bool = True) -> Dict:
      """Return the model's and base model's parameters.
      Args:
          deep: Whether to recursively return the base model's parameters.
      Returns
          Dict: The model's parameters.
      """
      
      params = {'base_model': self.base_model,
                'outcome_range': self.outcome_range,
                'rules': self.rules
                }

      params.update(self.base_model.get_params(deep=deep))
      return params
    
  def set_params(self, **params):
      """Sets parameters for the model and base model.
      Args:
          **params: Optional keyword arguments.
      """
                
      parameters = params
      param_keys = parameters.keys()
      
      if 'base_model' in param_keys:
          value = parameters.pop('base_model')
          self.base_model = value

In [25]:
""" Code """
def get_start_end(data, full_dd):
    # Get approximate index for sentence
    approx_idx = None
    words = data.split(" ")
    for w in words:
        if len(w) > 0:  
            try: 
                fwd_idx = full_dd.find(w)
                rev_idx = full_dd.rfind(w)
                if fwd_idx == rev_idx:
                    approx_idx = fwd_idx
                    break
            except:
                pass

    approx_lower = (approx_idx - len(data))
    approx_upper = (approx_idx + len(data))

    # find starting and end indices
    i=0
    while full_dd[approx_lower:approx_upper].find(words[i]) == -1:
            i+=1

    start = full_dd[approx_lower:approx_upper].find(words[i])
    if start == -1: 
        raise Exception
    start += approx_lower

    i=len(words)-1
    while full_dd[approx_lower:approx_upper].rfind(words[i]) == -1:
            i-=1

    end = full_dd[approx_lower:approx_upper].rfind(words[i])
    if end == -1: 
        raise Exception
    end += approx_lower
    end += len(words[i])
    
#     print(f"Data: {data}")
#     print(f"Daily Data: {full_dd[start:end]}")
    
    return start, end

In [26]:
with open("hybrid_model_gen_train.pkl", "rb") as f:
    model = pickle.load(f)

/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/conda/envs/opennotes/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [28]:
!pip install anvil-uplink 
import anvil.server
anvil.server.connect("MOTXDGUEXATA45F7IWO43RPN-ZY7LJ3KQDIXNU7FW") 

  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Disconnecting from previous connection first...
Connecting to wss://anvil.works/uplink
Anvil websocket closed (code 1000, reason=b'')
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [29]:
hadm_ids[:10]

[155131.0,
 129414.0,
 133623.0,
 197325.0,
 186291.0,
 180836.0,
 154802.0,
 133857.0,
 166389.0,
 196357.0]

In [32]:
def _get_contradictions(hadm_id): 
    # Step 1: Process data for patient
    print(f"Processing patient {int(hadm_id)} data...")
    pat = Patient(hadm_id, notes_df, drug_df, lab_df, d_lab_df, \
                  med7_nlp, umls_nlp, rxnorm_nlp, umls_linker, rxnorm_linker, \
                  physician_only=True)

    # Step 2: Generate pairs for this patient
    print(f"Generating candidate data pairs...")
    df, data_inst_pairs = generate_data_pairs(pat)

    # Step 3: Building feature set for pairs
    total_features = []
    for pair_id in tqdm(range(len(data_inst_pairs))):
        (s1, s2), _ = data_inst_pairs[pair_id]
        features = get_feature_df_mimic(s1, s2, hadm_id, pair_id)
        total_features.append(features)

    cols = \
    ["neg_check_umls", "check_umls", "check_med7", \
         "sentence_1", "sentence_2", "dep_sim", \
         "hadm_id", "conflict_type", "pair_id"]
    mimic_total_features_df = pd.DataFrame(total_features, columns=cols)

    # Step 4: Predicting contradictions
    mimic_X = mimic_total_features_df[['check_umls', 'neg_check_umls', "check_med7", "dep_sim"]]
    predicted_contradiction = model.predict(mimic_X)
    
    # Step 5: Building contradiction dictionary for Anvil
    #         Dictionary, {“lab”: (contradiction 1, contradiction 2, …), “prescription”: …} 
    #         contradiction i = (sentence 1, sentence 2, conflict type, 
    #                            data type 1, data type 2, full note 1, full note 2, 
    #                            time 1, time 2,
    #                            start index 1, end index 1, start index 2, end index 2, same note?) 
    # Iterate over the data pairs and add to output
    output = {"diagnosis": [], "prescription": [], "lab": []}

    for i in tqdm(range(len(data_inst_pairs))):
        (s1, s2), _ = data_inst_pairs[i]

        feats = mimic_total_features_df.iloc[i]
        label = predicted_contradiction[i]

        if label != 1: # skip anything not a contradiction
            continue

        is_same_dd = (s1.dailydata == s2.dailydata)
        time1 = s1.dailydata.time.strftime("%m/%d/%Y, %H:%M:%S")
        time2 = s2.dailydata.time.strftime("%m/%d/%Y, %H:%M:%S")

        # get start and end indices for sentences
        # if fail, continue
        if s1.type == "sentence":
            try: # try getting start and end indices
                start1, end1 = get_start_end(s1.txt, s1.dailydata.txt)
            except Exception:
                continue
        else:
            start1, end1 = None, None

        if s2.type == "sentence":
            try: # try getting start and end indices
                start2, end2 = get_start_end(s2.txt, s2.dailydata.txt)
            except Exception:
                continue
        else:
            start2, end2 = None, None

    #         contradiction_info = (
    #             s1.txt, s2.txt, feats.conflict_type, \
    #             s1.type, s2.type, s1.dailydata.txt, s2.dailydata.txt, \
    #             time1, time2, \
    #             start1, end1, start2, end2, is_same_dd
    #         )
        if s1.type == "sentence": s1_dd = s1.dailydata.txt
        else:                     s1_dd = None
        if s2.type == "sentence": s2_dd = s2.dailydata.txt
        else:                     s2_dd = None

        if "med" in feats.conflict_type: 
            contradiction_info = (
                s1.txt, s2.txt, "prescription", \
                s1.type, s2.type, s1_dd, s2_dd, \
                time1, time2, \
                start1, end1, start2, end2, is_same_dd
            )
            output["prescription"].append(contradiction_info)
            print("Adding prescription")
        elif "test" in feats.conflict_type:
            contradiction_info = (
                s1.txt, s2.txt, "lab", \
                s1.type, s2.type, s1_dd, s2_dd, \
                time1, time2, \
                start1, end1, start2, end2, is_same_dd
            )
            output["lab"].append(contradiction_info)
            print("Adding lab")
        elif "diagnosis" in feats.conflict_type:
            contradiction_info = (
                s1.txt, s2.txt, "diagnosis", \
                s1.type, s2.type, s1_dd, s2_dd, \
                time1, time2, \
                start1, end1, start2, end2, is_same_dd
            )
            output["diagnosis"].append(contradiction_info)
            print("Adding diagnosis")

    return output

In [34]:
# # Caching some outputs for speed
# for hadm_id in hadm_ids[10:20]:
#     output = get_contradictions(hadm_id)
#     with open(f"{int(hadm_id)}.pkl", "wb") as f:
#         pickle.dump(output, f)

In [35]:
import anvil.server
anvil.server.connect("MOTXDGUEXATA45F7IWO43RPN-ZY7LJ3KQDIXNU7FW")

In [ ]:
@anvil.server.callable 
def get_contradictions(hadm_id):
    try:
        with open(f"{hadm_id}.pkl", "rb") as f:
            output = pickle.load(f)
    except:
        output = _get_contradictions(hadm_id)
        
    return output

anvil.server.wait_forever() 

# OTHER: Getting History + Allergy Information - @Sharon, you can ignore everytihng below

In [ ]:
# todo: 
# - DONE function to re-process all data from Patient instance -- pat.process_notes(); pat.process_by_date()
# - function to update Note -- should update dataframe of patient directly
#   - can go back to dataframe, but can't map tokenized sentence to original note in df -- todo
#   - function to update tokenized sentence
# - later: function to update original dataframe from patient dataframe

import re

def get_section(regex_dict, txt):
    """ Given a dictionary of start and end regex's for a
        particular section, gets the start and endpoint of 
        section in the text and returns indices. 
        Returns None if section does not exist.
    """
    try:
        start    = re.search(regex_dict["start"], txt).start()
        end      = re.search(regex_dict["end"],   txt).start()
    except AttributeError:
        start, end = None, None
    
    return start, end

note = pat.notes[4]

# Sections to store 
# note: most of these sections have already been removed,
#       but if they haven't might have to remove then 
#       reprocess everything
allergy_regex = {"start": "Allergies:",
                 "end":   "Last dose of Antibiotics:"}
history_regex = {"start": "Past medical history:",
                 "end":   "Other:"}

allergy_start, allergy_end = get_section(allergy_regex, note.txt)
history_start, history_end = get_section(history_regex, note.txt)

pt_allergies = "" if allergy_start is None else note.txt[allergy_start:allergy_end]
pt_histories = "" if history_start is None else note.txt[history_start:history_end]

print("******** Allergies ********")
print(pt_allergies[:100])
print("******** Histories ********")
print(pt_histories[:100])